<h2>Credit Default Analysis</h2>

<b>Problem Explained</b>

While working with Credit One they have asked us to analyze their customers who have been defaulting on loan payments. They have been realizing that over the years more customers have been defaulting and they want to understand if we can find any patterns in order to find a way to reduce the defaults and reduce the risk of losing business.

On this first part we will be analysing the data to understand what we have in hands. Understanding the customer is fundamental to understand why they default. 

Our data is a dataframe with 30,000 customers with defaulted payments in Taiwan with information which includes: 
Gender, education, marital status, amount of credit given, history of past payments, amount of bill statement and amount of previous payment. 

<h3>1. Data Preparation</h3>

In [63]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

In [64]:
credit = pd.read_csv('default of credit card clients.csv', header =1)
#The first rows of the data frame
credit.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [65]:
#Checking the data types of the variables
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
ID                            30000 non-null int64
LIMIT_BAL                     30000 non-null int64
SEX                           30000 non-null int64
EDUCATION                     30000 non-null int64
MARRIAGE                      30000 non-null int64
AGE                           30000 non-null int64
PAY_0                         30000 non-null int64
PAY_2                         30000 non-null int64
PAY_3                         30000 non-null int64
PAY_4                         30000 non-null int64
PAY_5                         30000 non-null int64
PAY_6                         30000 non-null int64
BILL_AMT1                     30000 non-null int64
BILL_AMT2                     30000 non-null int64
BILL_AMT3                     30000 non-null int64
BILL_AMT4                     30000 non-null int64
BILL_AMT5                     30000 non-null int64
BILL_AMT6               

In [66]:
#Description of the variables
credit.describe()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,...,30000.000000,30000.000000,30000.000000,30000.000000,3.000000e+04,30000.00000,30000.000000,30000.000000,30000.000000,30000.000000
mean,15000.500000,167484.322667,1.603733,1.853133,1.551867,35.485500,-0.016700,-0.133767,-0.166200,-0.220667,...,43262.948967,40311.400967,38871.760400,5663.580500,5.921163e+03,5225.68150,4826.076867,4799.387633,5215.502567,0.221200
std,8660.398374,129747.661567,0.489129,0.790349,0.521970,9.217904,1.123802,1.197186,1.196868,1.169139,...,64332.856134,60797.155770,59554.107537,16563.280354,2.304087e+04,17606.96147,15666.159744,15278.305679,17777.465775,0.415062
min,1.000000,10000.000000,1.000000,0.000000,0.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,...,-170000.000000,-81334.000000,-339603.000000,0.000000,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000
25%,7500.750000,50000.000000,1.000000,1.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,2326.750000,1763.000000,1256.000000,1000.000000,8.330000e+02,390.00000,296.000000,252.500000,117.750000,0.000000
50%,15000.500000,140000.000000,2.000000,2.000000,2.000000,34.000000,0.000000,0.000000,0.000000,0.000000,...,19052.000000,18104.500000,17071.000000,2100.000000,2.009000e+03,1800.00000,1500.000000,1500.000000,1500.000000,0.000000
75%,22500.250000,240000.000000,2.000000,2.000000,2.000000,41.000000,0.000000,0.000000,0.000000,0.000000,...,54506.000000,50190.500000,49198.250000,5006.000000,5.000000e+03,4505.00000,4013.250000,4031.500000,4000.000000,0.000000
max,30000.000000,1000000.000000,2.000000,6.000000,3.000000,79.000000,8.000000,8.000000,8.000000,8.000000,...,891586.000000,927171.000000,961664.000000,873552.000000,1.684259e+06,896040.00000,621000.000000,426529.000000,528666.000000,1.000000


In [67]:
#Counts the NAs
credit.isna().sum()

ID                            0
LIMIT_BAL                     0
SEX                           0
EDUCATION                     0
MARRIAGE                      0
AGE                           0
PAY_0                         0
PAY_2                         0
PAY_3                         0
PAY_4                         0
PAY_5                         0
PAY_6                         0
BILL_AMT1                     0
BILL_AMT2                     0
BILL_AMT3                     0
BILL_AMT4                     0
BILL_AMT5                     0
BILL_AMT6                     0
PAY_AMT1                      0
PAY_AMT2                      0
PAY_AMT3                      0
PAY_AMT4                      0
PAY_AMT5                      0
PAY_AMT6                      0
default payment next month    0
dtype: int64

In [68]:
#Removes the column ID which is irrelevant for our analysis
credit = credit.drop(columns=['ID'])

In [69]:
#Assign AGE into four bins
credit['AGE'] = pd.cut(x=credit['AGE'], bins=[20, 35, 50, 65, 79], labels=[1,2,3,4])

In [70]:
#Confirming the change
credit['AGE'].value_counts()

1    16809
2    10922
3     2183
4       86
Name: AGE, dtype: int64

In [71]:
#Checking the data types of the variables
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 24 columns):
LIMIT_BAL                     30000 non-null int64
SEX                           30000 non-null int64
EDUCATION                     30000 non-null int64
MARRIAGE                      30000 non-null int64
AGE                           30000 non-null category
PAY_0                         30000 non-null int64
PAY_2                         30000 non-null int64
PAY_3                         30000 non-null int64
PAY_4                         30000 non-null int64
PAY_5                         30000 non-null int64
PAY_6                         30000 non-null int64
BILL_AMT1                     30000 non-null int64
BILL_AMT2                     30000 non-null int64
BILL_AMT3                     30000 non-null int64
BILL_AMT4                     30000 non-null int64
BILL_AMT5                     30000 non-null int64
BILL_AMT6                     30000 non-null int64
PAY_AMT1             

In [72]:
credit = credit.rename(columns={"default payment next month": "DEFAULT",})

In [73]:
#Since 4, 5, 6 and 0 are "others" lets replace all of them by 4
credit['EDUCATION'] = credit['EDUCATION'].replace([0,5,6], 4)
#And confirm the change
credit['EDUCATION'].value_counts()

2    14030
1    10585
3     4917
4      468
Name: EDUCATION, dtype: int64

In [74]:
#This is our final table after data cleaning 
credit.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,DEFAULT
0,20000,2,2,1,1,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
1,120000,2,2,2,1,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,90000,2,2,2,1,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,50000,2,2,1,2,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,50000,1,2,1,3,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


<h3> 2. Data Vizualization </h3>

In [75]:
header = redit.dtypes.index
print(header)

NameError: name 'redit' is not defined

In [ ]:
credit.groupby('SEX')['SEX'].count()

In [ ]:
sns.catplot('SEX', data=credit, kind='count', palette="muted")

In [ ]:
The dataset contains more 2= female than 1= male. 

In [ ]:
#Using seaborn to plot the count of clients in each education category
ec = sns.catplot('EDUCATION', data=credit, kind='count', aspect=1, palette="muted")
ec.set_xlabels('EDUCATION')

In [ ]:
# Number of men and women on each education
credit.groupby(['SEX', 'EDUCATION'])['SEX'].count()

In [ ]:
#Graph with with Education divided per Sex
g = sns.catplot('EDUCATION', data=credit, hue='SEX', kind='count', palette="muted")
g.set_xlabels('EDUCATION')

There are no strong diferences between male and female, but it seems to indicate the the female have higher education degree than male. 

In [ ]:
#Men and women in each of the age category
credit.groupby(['AGE','SEX'])['AGE'].count()

In [ ]:
#Using seaborn to group by men and women in each of the age category
a = sns.catplot('AGE', data=credit, hue='SEX', kind='count', palette="muted")
a.set_xlabels('AGE')

There are more women on the age bracket from 21 to 50. On the older bracket, above 50, the amount is identical. 

In [ ]:
#Default count   0=not default, 1=default
credit.groupby('DEFAULT')['DEFAULT'].count()

In [ ]:
#Using seaborn to count default
sns.catplot('DEFAULT', data=credit, kind='count', palette="muted")

Out of the whole customers analysed, 22% of them default. 

<h3> What affects the Default </h3>

In [ ]:
# Number of clients who are default grouped by sex. 
credit.groupby(['SEX', 'DEFAULT'])['SEX'].count()

24% of male default vs 21% of female

In [ ]:
#Graph with with Default divided per SEX
g = sns.catplot('SEX', data=credit, hue='DEFAULT', kind='count', palette="muted")
g.set_xlabels('SEX')

Blue = not default; Orange = default. 1 = Male, 2 = Female

In [ ]:
# Number of clients who are default grouped by sex. 
credit.groupby(['EDUCATION', 'DEFAULT'])['EDUCATION'].count()

Gradute 1 = 19% default; University 2 = 23,7%; High School = 25,15%; Others = 7%
Lower education means slight higher default rate

In [ ]:
#Graph with with Default divided per EDUCATION
g = sns.catplot('EDUCATION', data=credit, hue='DEFAULT', kind='count', palette="muted")
g.set_xlabels('EDUCATION')

In [ ]:
apl = sns.lmplot('AGE','DEFAULT',hue='AGE',data=credit)
apl.set(ylim=(0,0.4))

In [ ]:
#Graph with with Default divided per Marriage
sns.boxplot('MARRIAGE', data=credit, hue='DEFAULT', palette=["m", "g"])


In [ ]:
#Marital Status Factor 
sns.catplot('MARRIAGE', 'DEFAULT', data=credit, kind='point')

In [ ]:
0=Others, 1=Married, 2=Single, 3=Divorce
The single clientes are less prone to get divorced. 

In [ ]:
fig = sns.FacetGrid(credit, hue='DEFAULT', aspect=4)
fig.map(sns.kdeplot, 'LIMIT_BAL', shade=True)
largest = credit['LIMIT_BAL'].max()
fig.set(xlim=(0,largest))
fig.set(title='Distribution of given credit Grouped by Default')
fig.add_legend()

We can see clearly that the clients that ask for smaller credits have a greater possibility of default. 

In [ ]:
corrMat = credit.corr()
print(corrMat)

In [ ]:
#Covariance matrix and use heatmap to present it visually
covMat = credit.cov()
plt.figure(figsize = (10,5))
sns.heatmap(covMat)

<b>Conclusions</b>
The typical client that defaults is a married or divorced male with high school education which gets a small credit loan.  